In [1]:
import geemap
import ee

from ipyleaflet import basemaps
import datetime

# Air Pollution over Indonesia

Carbon Monoxide Data is obtained from Copernicus Atmosphere Monitoring Service (https://atmosphere.copernicus.eu/) overlaid with Coal Power Plants data from World Resource Institute (WRI). Units in mol/m^2 and values below 0.01 mol/m^2 are not shown on the map. This map is updated every 6 hours. For more information, please visit https://josefmtd.com/.

In [2]:
# Get the observed datetime
now = datetime.datetime.utcnow()
step = int(now.hour / 6)
obs = datetime.datetime(now.year, now.month, now.day, step * 6)

print('Datetime :', obs.strftime("%Y-%m-%d %H:%M UTC"))

Datetime : 2021-07-30 06:00 UTC


In [3]:
# Create a map service
Map = geemap.Map(center=(0,120), zoom=5,
    min_zoom=5, max_zoom=12,
    basemap=basemaps.CartoDB.DarkMatter,
    add_google_map=False
)

In [4]:
# Obtain PM2.5 data from Atmosphere Data Store API
# exported to Google Cloud Storage
fname = f'tcco_{obs.strftime("%Y%m%dT%H")}.tif'
var = ee.Image.loadGeoTIFF(f'gs://cams-neonet/{fname}')
data = var.resample().reproject(ee.Projection('EPSG:4326'), scale=1000)
data = data.updateMask(data.gt(0.01))

# Add Coal Power Plant from WRI's Global Power Plant Database
indo_power = ee.FeatureCollection('WRI/GPPD/power_plants') \
    .filter(ee.Filter.eq('country', 'IDN')) \
    .filter(ee.Filter.eq('fuel1', 'Coal'))

In [5]:
# Limit the area around Indonesia
bbox = ee.Geometry.BBox(80, -15, 160, 15)

# Visualization Parameters
vmin = 0
vmax = 0.05
palette = ['blue', 'yellow', 'red', 'black']

vis_params = {
    'min' : vmin,
    'max' : vmax,
    'palette' : palette,
    'opacity' : 0.5,
}

# Add Coal Power Plant data and PM2.5 data
Map.addLayer(data.clip(bbox), vis_params, 'PM2.5')
Map.addLayer(indo_power, {'color' : 'ff0000'}, 'Coal Power Plant')
Map.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax,
    layer_name='PM2.5')
Map.addLayerControl()
Map

Map(center=[0, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…